<div style="background-color: #FFDDDD; border-left: 5px solid red; padding: 10px; color: black;">
    <strong>Kernel:</strong> Python 3 (ipykernel)
</div>

![Meta Llama3.1 8b Instruct](https://developer-blogs.nvidia.com/wp-content/uploads/2024/04/dev-llama3-blog-1920x1080-1.png)

# 🚀 Deploy `meta-llama/Meta-Llama-3.1-8B-Instruct` on Amazon SageMaker

To start off, let's install some packages to help us through the notebooks

In [ ]:
%pip uninstall -q -y autogluon-multimodal autogluon-timeseries autogluon-features autogluon-common autogluon-core

In [ ]:
%pip install -Uq langchain==0.2.16 streamlit==1.38.0 wikipedia faiss-cpu opensearch-py==2.3.2 mlflow==2.13.2 sagemaker-mlflow==0.1.0 accelerate==0.27.2 huggingface_hub psutil pynvml numexpr==2.10.1 wikipedia==1.4.0 langchain_experimental==0.0.65 pydantic==2.9.1 py7zr==0.22.0 datasets==2.21.0 transformers==4.45.0 peft==0.12.0

In [ ]:
import os
import sagemaker
from sagemaker.djl_inference import DJLModel
from ipywidgets import Dropdown

import sys
sys.path.append(os.path.dirname(os.getcwd()))

from utilities.helpers import (
    pretty_print_html, 
    set_meta_llama_params,
    print_dialog,
    format_messages,
    write_eula
)

In [ ]:
session = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = session.default_bucket()

## License/EULA

#### Please review [Llama LICENSE](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/blob/main/LICENSE) before continuing!

In [ ]:
eula_dropdown = Dropdown(
    options=["True", "False"],
    value="False",
    description="**Please accept Llama 3.1 8B Instruct EULA to continue:**",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(eula_dropdown)

In [ ]:
llama_eula = f'{str(eula_dropdown.value.capitalize())}'
pretty_print_html(f"Your Llama 3.1 EULA attribute is set to 👉 {llama_eula}")

In [ ]:
_ = write_eula(llama_eula)

## Deploy Model to SageMaker Hosting

### Step 1: Get SageMaker LMI Container to host Llama

In [ ]:
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="djl-lmi", 
    region=session.boto_session.region_name, 
    version="0.29.0"
)
pretty_print_html(f"using image to host: {inference_image_uri}")

### Step 2: Deploy model using `DJLModel`

In [ ]:
inference_llm_config = {
    "HF_MODEL_ID": f"s3://{default_bucket}/sagemaker/models/base/llama3_1_8b_instruct/",
    "OPTION_MAX_MODEL_LEN": "4096",
    "OPTION_GPU_MEMORY_UTILIZATION": "0.8",
    "OPTION_ENABLE_STREAMING": "false",
    "OPTION_ROLLING_BATCH": "auto",
    "OPTION_MODEL_LOADING_TIMEOUT": "3600",
    # "OPTION_OUTPUT_FORMATTER": "jsonlines",
    "OPTION_PAGED_ATTENTION": "false",
    "OPTION_DTYPE": "fp16",
}

In [ ]:
model_name = "meta-llama31-8b-instruct"

lmi_model = sagemaker.Model(
    image_uri=inference_image_uri,
    env=inference_llm_config,
    role=role,
    name=model_name
)

In [ ]:
endpoint_name = f"{model_name}-endpoint"

predictor = lmi_model.deploy(
    initial_instance_count=1, 
    instance_type="ml.g5.2xlarge",
    endpoint_name=endpoint_name
)